In [1]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

/home/sharat/anaconda2/envs/tf-gpu-py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import *
from keras.datasets import imdb

In [3]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
17465344/17464789 [==============================] - 13s 1us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [37]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128,input_length=maxlen))
#model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.4))
#model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(CuDNNLSTM(128))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 80, 128)           2560000   
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 80, 64)            24640     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 40, 64)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 64)            0         
_________________________________________________________________
cu_dnnlstm_9 (CuDNNLSTM)     (None, 128)               99328     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 129       
Total params: 2,684,097
Trainable params: 2,684,097
Non-trainable params: 0
___________________________________________________

In [38]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 3s 110us/step
Test score: 0.34522889872550966
Test accuracy: 0.84924


In [32]:
y_test[0:5]

array([0, 1, 1, 0, 1])